# Python SQL notebook with Almighty Oracle

_Version: 2025-01-13_

# First steps


## Requirements

- Connect to Univie VPN ([HowTo](https://moodle.univie.ac.at/mod/book/view.php?id=19086186&chapterid=154722))
- Python – [https://www.python.org/downloads/](https://www.python.org/downloads/)


## OracleDB module

Before we can connect we need to install a additonal package.

The `python-oracledb` module to connect to the database: `pip  install oracledb`.

Documentation can be found at [https://python-oracledb.readthedocs.io/en/latest/](https://python-oracledb.readthedocs.io/en/latest/).

## Let's start connecting!

Connect to the **UniVie VPN**.

In [1]:
import oracledb

The Oracle Instant Client comes prepackaged in the dbs_python.zip.
So we just need to call it to use python-oracledb in **Thick** mode.

In [2]:
path_to_client = "instantclient"
oracledb.init_oracle_client(lib_dir=path_to_client)
# add here the path to the instantclient

### Set the credentials

The defaults are as follow:

- Username: [your username] (e.g. a01234567)  
- Password: [your password] (default-password: dbsXX where XX is the last two digits of the current year, e.g. 22)

**It is recommended to connect to Almighty, use `sqlplus`, log in and change the password with `passw`.**

In [3]:
username="a12028811"  #db user name
password="dbs25" #db password 
con_string="oracle19.cs.univie.ac.at:1521/orclcdb"

### Connect

Documentation can be found at [https://python-oracledb.readthedocs.io/en/latest/user_guide/connection_handling.html](https://python-oracledb.readthedocs.io/en/latest/user_guide/connection_handling.html).

In `cx_Oracle` the paramaters `encoding` and `nencoding` were supported to be passed to `oracledb.connect()`.
This is no longer the case!

In [4]:
#consider to use with in order to avoid open connections
connection = oracledb.connect(user=username,password=password,
                              dsn=con_string)
'''
with oracledb.connect(user=username,password=password, dsn=con_string) as connection:
    with connection.cursor() as cursor:
    ...
'''

'\nwith oracledb.connect(user=username,password=password, dsn=con_string) as connection:\n    with connection.cursor() as cursor:\n    ...\n'

Check if connection is now up!

In [5]:
# CLEAN UP OLD TABLE
try:
    connection.cursor().execute("drop table myTestTable")
    print("Old table dropped.")
except oracledb.DatabaseError as e:
    error, = e.args
    if error.code == 942:
        print("Table didn't exist yet, proceeding...")
    else:
        raise

Table didn't exist yet, proceeding...


In [6]:
#create a table
curs=connection.cursor()
create_stat="create table myTestTable(myId INTEGER)" #just an example: always use a PK
curs.execute(create_stat)
connection.commit()

In [7]:
#insert into
myIdValue=4
create_stat="insert into myTestTable (myId) VALUES (:myIdValue)"
curs.execute(create_stat,myIdValue=myIdValue)
connection.commit()

In [8]:
#select
curs.execute("select * from myTestTable")
res = curs.fetchall()

In [9]:
str(res)  #result should give '[(4,)]'

'[(4,)]'

In [10]:
[x[0] for x in res]

[4]

### Read from CSV

You need to install `pandas` from your CMD.
`python -m pip install pandas`

In [11]:
import pandas as pd

In [12]:
example_data=pd.read_csv("ExampleData.csv",sep=";")

In [13]:
example_data.head()

,Id,Name
0,9,Ann
1,10,Bob
2,11,Carol


In [ ]:
# DROP TABLE
try:
    curs.execute("drop table myTestTable2")
    print("Old myTestTable2 dropped.")
except oracledb.DatabaseError as e:
    if e.args[0].code == 942:
        print("Table didn't exist yet, proceeding.")
    else:
        raise

Old myTestTable2 dropped.


In [15]:
#create a table
create_stat="create table myTestTable2(myId INTEGER,myName VARCHAR(30))" #just an example: always use a PK
curs.execute(create_stat)
connection.commit()

In [16]:
def add_tuple(m_tuple):
    create_stat="insert into myTestTable2 (myId,myName) VALUES (:myId,:myName)"
    curs.execute(create_stat,myId=m_tuple.Id,myName=m_tuple.Name)
    connection.commit()

In [17]:
example_data.apply(lambda x: add_tuple(x),axis=1)  #use for instead of apply if that is easier for your!
print("done with inserting....")

done with inserting....


In [18]:
curs.execute("select * from myTestTable2")
res = curs.fetchall()
res # result should give [(9, 'Ann'), (10, 'Bob'), (11, 'Carol')]

[(9, 'Ann'), (10, 'Bob'), (11, 'Carol')]

### Close Connection

In [19]:
curs.close()
connection.close()